In [1]:
import math
import pylab
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F


seed =11
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)



class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

In [2]:
from collections import OrderedDict
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.L = 128
        self.D = 64
        self.K = 1
        self.feature_size = 24
        self.shared_layer_size = 512

        self.fc = nn.Sequential(
            nn.Linear(24, 128),
#             nn.Dropout(0.1),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
#             nn.Dropout(0.1),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Dropout(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(9728, 512), #166400
            nn.BatchNorm1d(512),
#             nn.ReLU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
#             nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
                               
    def forward(self, x):
        
        Y_prob=self.fc(x)
        Y_prob=self.fc2(Y_prob.reshape(Y_prob.size(0),-1))
        
        return Y_prob

In [3]:
from torch.optim.lr_scheduler import _LRScheduler
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [4]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [5]:
train_features=np.load("../extracted_features/FS/input_features_free_train.npy")
labels_stress_train=np.load("../extracted_features/FS/label_free_train.npy")


test_features=np.load("../extracted_features/FS/input_features_free_test.npy")
labels_stress_test=np.load("../extracted_features/FS/label_free_test.npy")


val_features=np.load("../extracted_features/FS/input_features_free_val.npy")
labels_stress_val=np.load("../extracted_features/FS/label_free_val.npy")


In [6]:
print(labels_stress_train.shape,labels_stress_train)

(1121,) [0. 0. 0. ... 1. 1. 1.]


In [7]:
print(train_features.shape,labels_stress_train.shape)
print(test_features.shape,labels_stress_test.shape)
print(val_features.shape,labels_stress_val.shape)

(1121, 19, 24) (1121,)
(190, 19, 24) (190,)
(199, 19, 24) (199,)


In [8]:
batch_size=50
train_data = Dataset(train_features, labels_stress_train)
test_data=Dataset(test_features,labels_stress_test)
val_data=Dataset(val_features,labels_stress_val)
train_data_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [9]:
for batch in val_data_loader:
    print(batch[1].shape)
    break
print(len(val_data_loader))

torch.Size([50])
4


In [10]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 50
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
model.apply(init_weights)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [11]:
# from torchsummary import summary
# summary(model.cuda(),(1,19,24))
# torch.save(model, "modelarchitect_final.pt")

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
# best_model = "./runs/"+ts+"/best_model_CNN_Cuda1.pth"
best_acc1 = 0
best_model1 = "./saved_models/best_model_dnn_fs"
modelname=[]
truth=[]
preds=[]

for it in range(epoch+1):
    model.train()
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=f1_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    

    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=f1_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        torch.save(model.state_dict(), best_model1+".pth")
        modelname.append(best_model1+".pth")
        best_acc1 = valacc1
        best_state = model.state_dict()
        print('Best validation accuracy1 ', best_acc1)
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(test_data_loader)*batch_size
    model.eval()    
    for minibatch in test_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=f1_score(truth,preds)
    val_loss /= total
    print('Test : Loss: {:.4f}, test acc1 : {:.4f}'.format(val_loss,valacc1))

EPOCH  0
Train : Loss: 0.0181, Train acc1 : 0.5438
Val : Loss: 0.0192, Val acc1 : 0.5736
Best validation accuracy1  0.5735849056603775
Test : Loss: 0.0210, test acc1 : 0.6246
EPOCH  1
Train : Loss: 0.0144, Train acc1 : 0.5997
Val : Loss: 0.0151, Val acc1 : 0.4151
Test : Loss: 0.0149, test acc1 : 0.6457
EPOCH  2
Train : Loss: 0.0140, Train acc1 : 0.5933
Val : Loss: 0.0144, Val acc1 : 0.4623
Test : Loss: 0.0155, test acc1 : 0.6307
EPOCH  3
Train : Loss: 0.0134, Train acc1 : 0.6203
Val : Loss: 0.0144, Val acc1 : 0.4205
Test : Loss: 0.0152, test acc1 : 0.6524
EPOCH  4
Train : Loss: 0.0136, Train acc1 : 0.6197
Val : Loss: 0.0149, Val acc1 : 0.4775
Test : Loss: 0.0154, test acc1 : 0.6400
EPOCH  5
Train : Loss: 0.0131, Train acc1 : 0.6223
Val : Loss: 0.0149, Val acc1 : 0.3902
Test : Loss: 0.0157, test acc1 : 0.6446
EPOCH  6
Train : Loss: 0.0136, Train acc1 : 0.6163
Val : Loss: 0.0139, Val acc1 : 0.3636
Test : Loss: 0.0155, test acc1 : 0.3185
EPOCH  7
Train : Loss: 0.0129, Train acc1 : 0.6258


In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.43 0.60 0.44 0.93 0.47 0.01


In [14]:
for i in range(len(modelname)):
    modeltest=Classifier()
    best_state=torch.load(modelname[i])
    modeltest.load_state_dict(best_state)
    modeltest.cuda()
    modeltest.eval()
    truth=[]
    preds=[]
    for minibatch in test_data_loader:
                X_test, Y1_test  = minibatch
                X_test=X_test.cuda()
                Y1_test=Y1_test.cuda()
                output_test = modeltest(X_test.float())
                output_test=output_test.squeeze(1)
                prediction = torch.ge(output_test, 0.5).float()
                truth.extend(Y1_test.tolist())
                preds.extend(prediction.tolist())
    acc=accuracy_score(truth,preds)
    # print(truth,preds)
    tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
    f1score=f1_score(truth, preds)
    precision=precision_score(truth, preds)
    recall=recall_score(truth,preds)
    roc=roc_auc_score(truth,preds)
    specificity=tn/(tn+fp)
    print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))

0.43 0.60 0.44 0.93 0.47 0.01
0.43 0.60 0.44 0.93 0.47 0.01


In [15]:
# torch.save(best_state, "./saved_model-baseline-free/DNN_best_free_speech_F1_52.pth")